In [1]:
import random
import numpy as np

In [11]:
# game simulation class
class Round():
    def __init__(self):
        self.p1_payoff = -1
        self.p2_payoff = -1
        self.pot = 2
    
    def check_conditions(self):
        return self.p1_payoff + self.p2_payoff == -self.pot

    def determine_winner(self):
        if self.p1_draw > self.p2_draw: #p1 wins
            self.p1_payoff += self.pot
            self.p2_payoff -= self.pot
        else: # p2 wins
            self.p1_payoff -= self.pot
            self.p2_payoff += self.pot
        
        # redistribute the pot (ASSUMPTION for stage game -- ante for next round is just paid back)
        self.p1_payoff += self.pot/2
        self.p2_payoff += self.pot/2
        self.pot = 0

        return (self.p1_payoff, self.p2_payoff)
    
    def simulate(self, x=1/2): 
        #initial draw
        self.p1_draw = random.uniform(0, 1)
        self.p2_draw = random.uniform(0, 1)

        p1_ante = 0
        p2_ante = 0
        #p1 conditional redraw to force median = 2/3
        if self.p1_draw <= 1/2 or self.p1_draw >= 5/6:
            self.p1_draw = max(random.uniform(0, 1), self.p1_draw)
            p1_ante = self.pot/2
            self.p1_payoff -= p1_ante# half pot draw
        
        #p2 redraw condition
        if self.p2_draw <= x or self.p2_draw >= 1-(x**2/(1+x)):
            # print(f'x, y: {x, x**2/(1+x)}')
            self.p2_draw = max(random.uniform(0, 1), self.p2_draw)
            p2_ante = self.pot/2
            self.p2_payoff -= p2_ante# half pot draw
    
        # update pot size
        self.pot += p1_ante + p2_ante

        # make sure pot is still right size
        self.check_conditions()

        # showdown conditions -- we play when our hand is above threshold
        threshold = 2/3
        return self.showdown(threshold)
    
    def showdown(self, threshold):
        # case 1: both play
        if self.p1_draw >= threshold and self.p2_draw >= threshold:
            return self.determine_winner()
        # case 2: only p1 plays
        elif self.p1_draw >= threshold and self.p2_draw < threshold:
            return (self.p1_payoff+self.pot, self.p2_payoff)
        #case 3: only p2 plays
        elif self.p1_draw < threshold and self.p2_draw >= threshold:
            return (self.p1_payoff, self.p2_payoff+self.pot)
        #case 4: neither play
        else:
            return self.showdown(threshold/2)

In [17]:
p1_total = 0
p2_total = 0
for i in range(100000):
    curr_round = Round()
    p1, p2 = curr_round.simulate()
    p1_total += p1
    p2_total += p2

print(f'(p1 total, p2 total): {(p1_total, p2_total)}')

(p1 total, p2 total): (8.0, -8.0)


In [100]:
#simulate on different x thresholds
x_vals = np.linspace(0, 1, 10)
p1_totals = [0 for i in range(len(x_vals))]
p2_totals = [0 for i in range(len(x_vals))]

for ind, x in enumerate(x_vals):
    for i in range(100000):
        curr_round = Round()
        p1, p2 = curr_round.simulate(x)
        p1_totals[ind] += p1
        p2_totals[ind] += p2


print(f'(p1 total, p2 total): {(p1_totals, p2_totals)}')
print(x_vals)

(p1 total, p2 total): ([8091.5, 5097.0, 2960.0, 1400.0, 115.5, 686.0, 4694.0, 5590.0, 4408.5, 4876.5], [-8091.5, -5097.0, -2960.0, -1400.0, -115.5, -686.0, -4694.0, -5590.0, -4408.5, -4876.5])
[0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]
